In [1]:
#pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.datasets import mnist
import keras_tuner
import keras
import numpy as np

In [3]:
# the data, shuffled and split between train and test sets 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
print("Number of training examples :", X_train.shape[0], "and each image is of shape (%d, %d)"%(X_train.shape[1], X_train.shape[2]))
print("Number of training examples :", X_test.shape[0], "and each image is of shape (%d, %d)"%(X_test.shape[1], X_test.shape[2]))

Number of training examples : 60000 and each image is of shape (28, 28)
Number of training examples : 10000 and each image is of shape (28, 28)


In [5]:
X_train = X_train/255
X_test = X_test/255

In [6]:
Y_train = utils.to_categorical(y_train, 10) 
Y_test = utils.to_categorical(y_test, 10)

In [7]:
import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation,Dropout
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten,AveragePooling2D


In [8]:
hp=keras_tuner.HyperParameters()

In [9]:
model=Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),strides=(2,2), activation='relu',input_shape=(28,28,1)))
model_type = hp.Choice("model_type", ["max", "avg"])
model_type = hp.Choice("model_type", ["max", "avg"])
with hp.conditional_scope("model_type", ["max"]):
            if model_type == "max":
                model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))
                model.add(Conv2D(16, kernel_size=(4,4),strides=(2,2), activation='relu'))
                model.add(MaxPooling2D(pool_size=(4,4),strides=(2,2)))
with hp.conditional_scope("model_type", ["avg"]):
            if model_type == "avg":
                model.add(AveragePooling2D(pool_size=(2,2),strides=(1,1)))
                model.add(Conv2D(16, kernel_size=(4,4),strides=(2,2), activation='relu'))
                model.add(AveragePooling2D(pool_size=(4,4),strides=(2,2)))

model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(10, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(hp.Float(
"learning_rate",
min_value=0.01,
max_value=0.15,
step=10,
sampling="log")),
     metrics=['accuracy'])
model.fit(X_train,Y_train,
batch_size=hp.Int(
"batch_size",
min_value=64,
max_value=512,
step=2,
sampling="log"),
               epochs=5,
               verbose=1,
               validation_data=(X_test,Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy:', score[1])

Epoch 1/5
938/938 [==============================] - 13s 13ms/step - loss: 2.3023 - accuracy: 0.1098 - val_loss: 2.3021 - val_accuracy: 0.1135
Epoch 2/5
938/938 [==============================] - 12s 12ms/step - loss: 2.3024 - accuracy: 0.1109 - val_loss: 2.3013 - val_accuracy: 0.1135
Epoch 3/5
938/938 [==============================] - 12s 13ms/step - loss: 2.3023 - accuracy: 0.1102 - val_loss: 2.3015 - val_accuracy: 0.1135
Epoch 4/5
934/938 [============================>.] - ETA: 0s - loss: 2.3020 - accuracy: 0.1098